In [1]:
import h5py
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('/home/amber/multitask_RNA/data_generation')
import utils
import numpy as np
import nucleotide_transformer
import haiku as hk
import jax
import jax.numpy as jnp
from nucleotide_transformer.pretrained import get_pretrained_model
from tqdm import tqdm
model_name = '500M_1000G'

In [2]:
data_file = h5py.File('/home/amber/multitask_RNA/data/lenti_MPRA/WTC11_data.h5', 'r')
sequence = data_file['seq'][()]
target = data_file['mean'][()]

In [3]:
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    mixed_precision=False,
    embeddings_layers_to_save=(24,),
    attention_maps_to_save=(),
    max_positions=41,
)
forward_fn = hk.transform(forward_fn)

In [4]:
#get embedding per input sequence
random_key = jax.random.PRNGKey(0)
N,  = sequence.shape
lenti_embed=[]
for i in tqdm(range(N)):
    seq = sequence[i].decode()
    token_out = tokenizer.batch_tokenize([seq])
    token_id = [b[1] for b in token_out]
    seq_pair = jnp.asarray(token_id,dtype=jnp.int32)
    outs = forward_fn.apply(parameters, random_key, seq_pair)
    lenti_embed.append(outs['embeddings_24'][0])


  0%|          | 35/55989 [00:21<9:34:25,  1.62it/s]


KeyboardInterrupt: 

In [ ]:
out_embed = h5py.File('/home/amber/multitask_RNA/data/lenti_MPRA/WTC11_data_embed.h5', 'w')
out_embed.create_dataset('embed', data=np.array(lenti_embed))
out_embed.create_dataset('target', data=np.array(target))